# Notebook Iteracion 3

 Modelado

### Objetivo:
Realizar la ejecución de diferentes tipos de algoritmos de machine larning con el fin de encontrar un modelo que supere las metricas obtenidas por la línea base en las iteraciones previas:
- Regresión Lineal
- Regresion Lasso
- Árbol de decisión
- Bosques aleatorios
- Maquinas de soporte vectorial

In [1]:
# Funciones generales
import sys
sys.path.append('../')
# A medida que avanzan las iteraciones se crearan mas funcinoes transversales
from funciones import *

# Tuneo de hiperparámetros
from sklearn.model_selection import ParameterGrid

Lectura de los datos resultantes de la iteración 2

In [2]:
import os 
# get current directory
path = os.getcwd()
# parent directory
path = os.path.abspath("../")
data_model = pd.read_csv(path + "/iteracion_2/datos_iteracion_2.csv")
data_model.head()

,precio,zona,barrio_sector,baños_familiares,area_bruta,numero_niveles,parqueaderos,alcobas_familiares,estrato,area_total,...,closet_de_linos,biblioteca,parqueadero_visitantes,gimnasio,piscina,salon_social,dispositivos_automatizacion,alarma,tipo_cocina_freq,tipo_pisos_freq
0,1100000.0,centro,castilla,1.0,95.0,1.0,0.0,3.0,0.0,95.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.861534,0.596709
1,950000.0,centro,el salvador,1.0,70.0,1.0,0.0,2.0,3.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.081652,0.596709
2,970000.0,centro,los angeles,1.0,38.0,1.0,0.0,1.0,4.0,38.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.081652,0.596709
3,1400000.0,centro,prado,1.0,50.0,1.0,1.0,2.0,0.0,50.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.861534,0.596709
4,800000.0,centro,12 de octubre,1.0,92.0,1.0,0.0,3.0,2.0,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.861534,0.596709


In [3]:
imprimir_dimensiones(data_model)

Numero de muestras: 2892, Número de columnas: 39


Eliminación de las variables por distribución y correlación (resultado de la iteración 2)

In [4]:
# Variables a descartar por correlación con variable final
columns_disc_corr = [
    "numero_niveles",
    "zona_ropas",
    "camaras_cctv",
    "cancha_polideportiva",
    "cancha_squash",
    "zona_bbq",
    "patio",
    "aire_acondicionado",
    "jacuzzi",
    "red_de_Gas",
    "terraza",
    "dispositivos_automatizacion",
    "alarma",
    "area_total",
    "porteria_24_7"
]

Por ahora no se usarán las variables de zona o barrio, por tanto, también se eliminan

In [5]:
columns_disc_corr.extend(['zona', 'barrio_sector'])

In [6]:
data_model = data_model.drop(columns=columns_disc_corr)
imprimir_dimensiones(data_model)

Numero de muestras: 2892, Número de columnas: 22


**Modelado**

Para cada tipo de modelo se realizara multiples ejecuciones que constan de la variación de hiperparámetros. Estos resultados seran tabulados y almacenados en un archivo csv identificado por cada tipo de modelo. Al final se presenta un resumen de los mejores resultados.

Las métricas evaluadas se ejecutaran en los conjuntos de entrenamiento y prueba con el fin de evitar el sobre entrenamiento, para esto se calcula la diferencia entre R2 entrnamiento y R2 prueba, esta no debe ser superior a 0.05

**Dvisión de los datos:** Entrenamiento 75%, Test 25%

In [7]:
# Separación de las variables predictoras y la variable respuesta
X = data_model.drop(columns=['precio'])
Y = data_model['precio']

In [8]:
X_est = estandarizar(X)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X_est, Y, test_size = 0.30, train_size = 0.70, random_state = 17)
print("Tamaño de los Datos de Entrenamiento = ", x_train.shape)
print("Tamaño de los Datos de Validación = ", x_test.shape)
print("Tamaño del Vector de Clases de Entrenamiento = ", y_train.shape)
print("Tamaño del Vector de Clases de Prueba = ", y_test.shape)

Tamaño de los Datos de Entrenamiento =  (2024, 21)
Tamaño de los Datos de Validación =  (868, 21)
Tamaño del Vector de Clases de Entrenamiento =  (2024,)
Tamaño del Vector de Clases de Prueba =  (868,)


**Regresión Lineal**

In [10]:
# Grilla de hiperparámetros
param_grid_lr = ParameterGrid({
    'fit_intercept' : [False, True]
})
# Ejecuciones del modelo
ejecutar_modelo(model = "LinearRegression", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_lr, 
                filename='LinearRegression')


Modelo: {'fit_intercept': False}
Modelo: {'fit_intercept': True}


**Regresión Lasso**

In [11]:
# Grilla de hiperparámetros
param_grid_lrl = ParameterGrid({
    'alpha' : [0.05,0.5,1,1.5,2,3],
    'max_iter': [800, 1000, 1500, 2000],
})
# Ejecuciones del modelo
ejecutar_modelo(model = "Lasso", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_lrl, 
                filename='Lasso')

Modelo: {'alpha': 0.05, 'max_iter': 800}
Modelo: {'alpha': 0.05, 'max_iter': 1000}
Modelo: {'alpha': 0.05, 'max_iter': 1500}
Modelo: {'alpha': 0.05, 'max_iter': 2000}
Modelo: {'alpha': 0.5, 'max_iter': 800}
Modelo: {'alpha': 0.5, 'max_iter': 1000}
Modelo: {'alpha': 0.5, 'max_iter': 1500}
Modelo: {'alpha': 0.5, 'max_iter': 2000}
Modelo: {'alpha': 1, 'max_iter': 800}
Modelo: {'alpha': 1, 'max_iter': 1000}
Modelo: {'alpha': 1, 'max_iter': 1500}
Modelo: {'alpha': 1, 'max_iter': 2000}
Modelo: {'alpha': 1.5, 'max_iter': 800}
Modelo: {'alpha': 1.5, 'max_iter': 1000}
Modelo: {'alpha': 1.5, 'max_iter': 1500}
Modelo: {'alpha': 1.5, 'max_iter': 2000}
Modelo: {'alpha': 2, 'max_iter': 800}
Modelo: {'alpha': 2, 'max_iter': 1000}
Modelo: {'alpha': 2, 'max_iter': 1500}
Modelo: {'alpha': 2, 'max_iter': 2000}


Modelo: {'alpha': 3, 'max_iter': 800}
Modelo: {'alpha': 3, 'max_iter': 1000}
Modelo: {'alpha': 3, 'max_iter': 1500}
Modelo: {'alpha': 3, 'max_iter': 2000}


**Árbol de decisión**

In [12]:
# Grilla de hiperparámetros
param_grid_dt = ParameterGrid({
    'splitter' : ['random', 'best'],
    'max_depth' : [3, 5, 7, 10, 15],
    'min_samples_split' : [2, 3, 5],
    'min_samples_leaf' : [1,3, 5],
    'max_features' : [None,'sqrt', 'log2'],
    'ccp_alpha':np.linspace(0, 5, 10)
})
# Ejecuciones del modelo
ejecutar_modelo(model = "DecisionTreeRegressor", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_dt, 
                filename='DecisionTreeRegressor')

Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'random'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'best'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'random'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'splitter': 'random'}
Modelo: {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': None, 'mi

**Random Forest**

In [13]:
# Grilla de hiperparámetros
param_grid_rf = ParameterGrid(
                {
                 'n_estimators': [40, 60, 80, 100, 150, 200],
                 'min_samples_split' : [2, 3, 5],
                 'min_samples_leaf' : [1,3, 5],
                 'max_features': [3,5,7,10],
                 'max_depth'   : [3, 5, 7, 10, 15]
                }
             )
# Ejecuciones del modelo
ejecutar_modelo(model = "RandomForestRegressor", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_rf, 
                filename='RandomForestRegressor')

Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 40}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 60}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 80}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 40}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 60}
Modelo: {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 80

**Máquinas de soporte vectorial**

In [14]:
# Grilla de hiperparámetros
param_grid_svm = ParameterGrid(
                {
                 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                 'C' : [100, 500, 5000, 10000, 150000, 20000],
                 'degree' : [2, 3, 4, 5],
                }
             )
# Ejecuciones del modelo
ejecutar_modelo(model = "SVR", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_svm, 
                filename='SVR')

Modelo: {'C': 100, 'degree': 2, 'kernel': 'linear'}
Modelo: {'C': 100, 'degree': 2, 'kernel': 'poly'}
Modelo: {'C': 100, 'degree': 2, 'kernel': 'rbf'}
Modelo: {'C': 100, 'degree': 2, 'kernel': 'sigmoid'}
Modelo: {'C': 100, 'degree': 3, 'kernel': 'linear'}
Modelo: {'C': 100, 'degree': 3, 'kernel': 'poly'}
Modelo: {'C': 100, 'degree': 3, 'kernel': 'rbf'}
Modelo: {'C': 100, 'degree': 3, 'kernel': 'sigmoid'}
Modelo: {'C': 100, 'degree': 4, 'kernel': 'linear'}
Modelo: {'C': 100, 'degree': 4, 'kernel': 'poly'}
Modelo: {'C': 100, 'degree': 4, 'kernel': 'rbf'}
Modelo: {'C': 100, 'degree': 4, 'kernel': 'sigmoid'}
Modelo: {'C': 100, 'degree': 5, 'kernel': 'linear'}
Modelo: {'C': 100, 'degree': 5, 'kernel': 'poly'}
Modelo: {'C': 100, 'degree': 5, 'kernel': 'rbf'}
Modelo: {'C': 100, 'degree': 5, 'kernel': 'sigmoid'}
Modelo: {'C': 500, 'degree': 2, 'kernel': 'linear'}
Modelo: {'C': 500, 'degree': 2, 'kernel': 'poly'}
Modelo: {'C': 500, 'degree': 2, 'kernel': 'rbf'}
Modelo: {'C': 500, 'degree': 2, '

**Consolidación de mejores resultados:**

Dada las ejecuciones de los modelos con la variación de sus hiperparámetros, se consolida algunos de los mejores resultados obtenidos de cada algoritmo. Cabe resaltar que algunas combinaciones de parámetros tienen los mismos resultados en métricas.

**Regresion Lineal**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | fit_intercept |
| --- | --- |  --- |  --- |  --- |  --- |  --- | 
| 0.74 | 0.71 | 1051860.93 | 1090624.48 | 0.28 | 0.28 | False |

**Regresion Lasso**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | Alpha | Max_iter |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- | 
| 0.74 | 0.71 | 1051096.22 | 1090480.00 | 0.28 | 0.29 | 3 | 1500 |

**Árbol de decisión**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | Ccp_alpha | Max_depth | Max_features | Min_samples | Min_samples_split | Splitter |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- |  --- | --- | --- | --- |
| 0.73 | 0.69 | 1077563.62 | 1140152.11 | 0.28 | 0.29 | 0 | 5 |  | 1 | 2 | random |

**Random Forest**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | max_depth | max_features | min_samples_leaf | min_samples_split | n_estimators |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- |  --- | --- | --- |
| 0.79 | 0.75 | 952251.42 | 1011693.32 | 0.24 | 0.25 | 5 | 10 | 5 | 5 | 40 |

**SVR**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | C | Degree | Kernel |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- | --- | 
| 0.71 | 0.68 | 1121700.44 | 1153141.45 | 0.25 | 0.26 | 150000 | 3 | linear |

**Conclusiones**

- Dados los resultados anteriores se obtienen mejoras para los algoritmos de __Random Forest__ y __Maquinas de soporte vectorial__, siendo estos los que mejoran el alcance a la métrica del negocio (MAPE <= 0.15). 
- Dado este punto donde se ejecutan algorítmos tradicionales, si bien se tiene una mejora, aún se está muy por encima de lograr el objetivo, por tanto, se considera la implementación de metodos mas robustos como lo es la estrategia de ensamble de modelos (boosting). Esta será la ejecución de la próxima iteración